In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split


from tensorflow.keras.preprocessing.image import ImageDataGenerator

import cv2


In [2]:
print(os.listdir('own_dataset/1d6/train'))

['5', '1', '4', '3', '6', '2']


In [3]:
train_path= ('own_dataset/1d6/train')
test_path= ('own_dataset/1d6/test')

In [4]:
batch_size_train=40  #,batch_size= batch_size_train
batch_size_valid=40
targetsize= 64
datagen=ImageDataGenerator(rotation_range=180,height_shift_range=0.1,rescale=1.0/255.0
                           ,zoom_range=[0.7,1.1],brightness_range=[0.4,1.2])
train_batches= datagen.flow_from_directory(train_path, target_size=(targetsize,targetsize), shuffle=True,
                                                                      classes=['5', '1', '4', '3', '6', '2'],
                                                                      batch_size= batch_size_train)
test_batches= datagen.flow_from_directory(test_path,  target_size=(targetsize,targetsize), shuffle=True,
                                                                     classes=['5', '1', '4', '3', '6', '2'],
                                                                     batch_size= batch_size_train)

Found 4254 images belonging to 6 classes.
Found 834 images belonging to 6 classes.


In [5]:
def plots(ims, figsize=(20,10), rows=1, interp= False, titles= None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims= ims.transpose((0,1,2,3))
    f= plt.figure(figsize=figsize)
    cols= len(ims)//rows if len(ims) %2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=12)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

In [6]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten

from tensorflow.keras.models import model_from_json


In [7]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 5e-4 *0.99**x)

In [8]:
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(targetsize,targetsize, 3)) )
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), padding='SAME', activation='relu'))
model.add(MaxPooling2D((2, 2)))


model.add(Conv2D(filters=128, kernel_size=(3,3), padding='SAME', activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding='SAME', activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding='SAME', activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(6, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 256)         2

In [9]:
model.compile(Adam(lr=.0002), loss='categorical_crossentropy', metrics= ['accuracy'])
history2= model.fit_generator(train_batches, steps_per_epoch= len(train_batches) , callbacks=[reduce_lr],
                             validation_data=test_batches, validation_steps= len(test_batches), 
                             epochs=500)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/500
107/107 [==============================] - 110s 1s/step - loss: 1.7917 - accuracy: 0.1824 - val_loss: 1.7890 - val_accuracy: 0.1882 - lr: 5.0000e-04
Epoch 2/500
107/107 [==============================] - 107s 1s/step - loss: 1.7894 - accuracy: 0.1869 - val_loss: 1.7880 - val_accuracy: 0.1918 - lr: 4.9500e-04
Epoch 3/500
107/107 [==============================] - 108s 1s/step - loss: 1.7899 - accuracy: 0.1866 - val_loss: 1.7893 - val_accuracy: 0.1871 - lr: 4.9005e-04
Epoch 4/500
107/107 [==============================] - 110s 1s/step - loss: 1.7875 - accuracy: 0.1944 - val_loss: 1.7769 - val_accuracy: 0.2026 - lr: 4.8515e-04
Epoch 5/500
107/107 [==============================] - 107s 997ms/step - loss: 1.7314 - accuracy: 0.2379 - val_loss: 1.6559 - val_accuracy: 0.2950 - lr: 4.8030e-04
Epoch 6/500
107/107 [==============================] - 108s 1s/step - loss: 1.6932 - accuracy: 0.2593 - val_loss: 1.

107/107 [==============================] - 106s 993ms/step - loss: 1.0184 - accuracy: 0.5914 - val_loss: 1.0236 - val_accuracy: 0.5971 - lr: 3.0865e-04
Epoch 50/500
107/107 [==============================] - 106s 986ms/step - loss: 1.0224 - accuracy: 0.5940 - val_loss: 1.1605 - val_accuracy: 0.5671 - lr: 3.0556e-04
Epoch 51/500
107/107 [==============================] - 107s 1s/step - loss: 1.1254 - accuracy: 0.5665 - val_loss: 1.2103 - val_accuracy: 0.5240 - lr: 3.0250e-04
Epoch 52/500
107/107 [==============================] - 109s 1s/step - loss: 1.1171 - accuracy: 0.5693 - val_loss: 1.0331 - val_accuracy: 0.6019 - lr: 2.9948e-04
Epoch 53/500
107/107 [==============================] - 107s 1s/step - loss: 1.0582 - accuracy: 0.5858 - val_loss: 1.3117 - val_accuracy: 0.5228 - lr: 2.9648e-04
Epoch 54/500
107/107 [==============================] - 106s 991ms/step - loss: 1.0650 - accuracy: 0.5882 - val_loss: 0.9772 - val_accuracy: 0.6235 - lr: 2.9352e-04
Epoch 55/500
107/107 [==========

107/107 [==============================] - 105s 978ms/step - loss: 1.0126 - accuracy: 0.6812 - val_loss: 1.2286 - val_accuracy: 0.6547 - lr: 1.8673e-04
Epoch 100/500
107/107 [==============================] - 104s 976ms/step - loss: 1.1989 - accuracy: 0.6646 - val_loss: 0.8633 - val_accuracy: 0.7194 - lr: 1.8486e-04
Epoch 101/500
107/107 [==============================] - 105s 979ms/step - loss: 1.0349 - accuracy: 0.6876 - val_loss: 0.9958 - val_accuracy: 0.6954 - lr: 1.8302e-04
Epoch 102/500
107/107 [==============================] - 108s 1s/step - loss: 1.0969 - accuracy: 0.6700 - val_loss: 1.0380 - val_accuracy: 0.7026 - lr: 1.8119e-04
Epoch 103/500
107/107 [==============================] - 106s 992ms/step - loss: 1.1176 - accuracy: 0.6761 - val_loss: 1.4517 - val_accuracy: 0.6115 - lr: 1.7937e-04
Epoch 104/500
107/107 [==============================] - 104s 968ms/step - loss: 1.1853 - accuracy: 0.6660 - val_loss: 1.2389 - val_accuracy: 0.6679 - lr: 1.7758e-04
Epoch 105/500
107/107

Epoch 149/500
107/107 [==============================] - 108s 1s/step - loss: 1.1360 - accuracy: 0.7318 - val_loss: 1.4074 - val_accuracy: 0.6799 - lr: 1.1297e-04
Epoch 150/500
107/107 [==============================] - 107s 997ms/step - loss: 1.2555 - accuracy: 0.7179 - val_loss: 1.0653 - val_accuracy: 0.7266 - lr: 1.1184e-04
Epoch 151/500
107/107 [==============================] - 106s 994ms/step - loss: 1.1948 - accuracy: 0.7118 - val_loss: 1.2846 - val_accuracy: 0.6978 - lr: 1.1073e-04
Epoch 152/500
107/107 [==============================] - 106s 990ms/step - loss: 1.3049 - accuracy: 0.7151 - val_loss: 1.4431 - val_accuracy: 0.7014 - lr: 1.0962e-04
Epoch 153/500
107/107 [==============================] - 106s 990ms/step - loss: 1.2720 - accuracy: 0.7149 - val_loss: 1.0151 - val_accuracy: 0.7326 - lr: 1.0852e-04
Epoch 154/500
107/107 [==============================] - 105s 984ms/step - loss: 1.1569 - accuracy: 0.7311 - val_loss: 1.0025 - val_accuracy: 0.7386 - lr: 1.0744e-04
Epoch 1

Epoch 199/500
107/107 [==============================] - 108s 1s/step - loss: 1.4428 - accuracy: 0.7330 - val_loss: 1.6056 - val_accuracy: 0.6966 - lr: 6.8350e-05
Epoch 200/500
107/107 [==============================] - 107s 1000ms/step - loss: 1.5354 - accuracy: 0.7212 - val_loss: 1.5003 - val_accuracy: 0.7266 - lr: 6.7667e-05
Epoch 201/500
107/107 [==============================] - 106s 994ms/step - loss: 1.4918 - accuracy: 0.7287 - val_loss: 1.8381 - val_accuracy: 0.7086 - lr: 6.6990e-05
Epoch 202/500
107/107 [==============================] - 106s 988ms/step - loss: 1.4934 - accuracy: 0.7257 - val_loss: 1.6348 - val_accuracy: 0.7218 - lr: 6.6320e-05
Epoch 203/500
107/107 [==============================] - 105s 982ms/step - loss: 1.4449 - accuracy: 0.7445 - val_loss: 1.7001 - val_accuracy: 0.7050 - lr: 6.5657e-05
Epoch 204/500
107/107 [==============================] - 107s 996ms/step - loss: 1.3375 - accuracy: 0.7546 - val_loss: 1.5142 - val_accuracy: 0.7422 - lr: 6.5000e-05
Epoch 

...

Epoch 298/500
107/107 [==============================] - 107s 996ms/step - loss: 1.8312 - accuracy: 0.7381 - val_loss: 1.8720 - val_accuracy: 0.7242 - lr: 2.5271e-05
Epoch 299/500
107/107 [==============================] - 106s 991ms/step - loss: 1.8321 - accuracy: 0.7379 - val_loss: 1.9192 - val_accuracy: 0.7374 - lr: 2.5018e-05
Epoch 300/500
107/107 [==============================] - 106s 990ms/step - loss: 1.9450 - accuracy: 0.7365 - val_loss: 1.9500 - val_accuracy: 0.7542 - lr: 2.4768e-05
Epoch 301/500
107/107 [==============================] - 107s 1000ms/step - loss: 1.9199 - accuracy: 0.7466 - val_loss: 2.0134 - val_accuracy: 0.7338 - lr: 2.4520e-05
Epoch 302/500
107/107 [==============================] - 106s 990ms/step - loss: 1.9792 - accuracy: 0.7268 - val_loss: 1.8566 - val_accuracy: 0.7542 - lr: 2.4275e-05
Epoch 303/500
107/107 [==============================] - 105s 984ms/step - loss: 1.9288 - accuracy: 0.7308 - val_loss: 2.0384 - val_accuracy: 0.7554 - lr: 2.4032e-05
Epo

107/107 [==============================] - 106s 989ms/step - loss: 2.0070 - accuracy: 0.7313 - val_loss: 2.0198 - val_accuracy: 0.7254 - lr: 1.5444e-05
Epoch 348/500
107/107 [==============================] - 108s 1s/step - loss: 2.0028 - accuracy: 0.7449 - val_loss: 2.1358 - val_accuracy: 0.7218 - lr: 1.5289e-05
Epoch 349/500
107/107 [==============================] - 107s 996ms/step - loss: 2.1125 - accuracy: 0.7301 - val_loss: 2.2428 - val_accuracy: 0.7290 - lr: 1.5136e-05
Epoch 350/500
107/107 [==============================] - 106s 994ms/step - loss: 2.0556 - accuracy: 0.7395 - val_loss: 2.1935 - val_accuracy: 0.7446 - lr: 1.4985e-05
Epoch 351/500
107/107 [==============================] - 106s 989ms/step - loss: 2.0784 - accuracy: 0.7417 - val_loss: 2.1314 - val_accuracy: 0.7326 - lr: 1.4835e-05
Epoch 352/500
107/107 [==============================] - 105s 985ms/step - loss: 2.0804 - accuracy: 0.7344 - val_loss: 2.1846 - val_accuracy: 0.7266 - lr: 1.4687e-05
Epoch 353/500
107/107

107/107 [==============================] - 107s 996ms/step - loss: 2.0685 - accuracy: 0.7452 - val_loss: 2.2079 - val_accuracy: 0.7326 - lr: 9.4378e-06
Epoch 397/500
107/107 [==============================] - 109s 1s/step - loss: 2.1742 - accuracy: 0.7431 - val_loss: 2.0226 - val_accuracy: 0.7518 - lr: 9.3434e-06
Epoch 398/500
107/107 [==============================] - 107s 999ms/step - loss: 2.1586 - accuracy: 0.7351 - val_loss: 2.2845 - val_accuracy: 0.7278 - lr: 9.2500e-06
Epoch 399/500
107/107 [==============================] - 105s 984ms/step - loss: 2.0922 - accuracy: 0.7365 - val_loss: 2.2844 - val_accuracy: 0.7398 - lr: 9.1575e-06
Epoch 400/500
107/107 [==============================] - 106s 986ms/step - loss: 2.1161 - accuracy: 0.7334 - val_loss: 2.2164 - val_accuracy: 0.7446 - lr: 9.0659e-06
Epoch 401/500
107/107 [==============================] - 106s 993ms/step - loss: 2.2735 - accuracy: 0.7280 - val_loss: 2.3107 - val_accuracy: 0.7338 - lr: 8.9753e-06
Epoch 402/500
107/107

107/107 [==============================] - 107s 1s/step - loss: 2.0696 - accuracy: 0.7412 - val_loss: 2.3831 - val_accuracy: 0.7278 - lr: 5.7676e-06
Epoch 446/500
107/107 [==============================] - 108s 1s/step - loss: 2.1579 - accuracy: 0.7358 - val_loss: 2.4699 - val_accuracy: 0.7098 - lr: 5.7099e-06
Epoch 447/500
107/107 [==============================] - 105s 986ms/step - loss: 2.0147 - accuracy: 0.7426 - val_loss: 2.4635 - val_accuracy: 0.7218 - lr: 5.6528e-06
Epoch 448/500
107/107 [==============================] - 103s 967ms/step - loss: 2.1189 - accuracy: 0.7351 - val_loss: 2.2302 - val_accuracy: 0.7230 - lr: 5.5963e-06
Epoch 449/500
107/107 [==============================] - 105s 984ms/step - loss: 2.1789 - accuracy: 0.7370 - val_loss: 2.2219 - val_accuracy: 0.7530 - lr: 5.5403e-06
Epoch 450/500
107/107 [==============================] - 105s 979ms/step - loss: 2.0995 - accuracy: 0.7402 - val_loss: 2.1800 - val_accuracy: 0.7314 - lr: 5.4849e-06
Epoch 451/500
107/107 [=

107/107 [==============================] - 108s 1s/step - loss: 2.2474 - accuracy: 0.7367 - val_loss: 2.2612 - val_accuracy: 0.7470 - lr: 3.5247e-06
Epoch 495/500
107/107 [==============================] - 106s 989ms/step - loss: 2.0715 - accuracy: 0.7464 - val_loss: 2.4015 - val_accuracy: 0.7206 - lr: 3.4894e-06
Epoch 496/500
107/107 [==============================] - 104s 976ms/step - loss: 2.3660 - accuracy: 0.7247 - val_loss: 2.2027 - val_accuracy: 0.7290 - lr: 3.4545e-06
Epoch 497/500
107/107 [==============================] - 103s 965ms/step - loss: 2.0783 - accuracy: 0.7379 - val_loss: 2.4881 - val_accuracy: 0.7122 - lr: 3.4200e-06
Epoch 498/500
107/107 [==============================] - 106s 995ms/step - loss: 2.3293 - accuracy: 0.7250 - val_loss: 2.6133 - val_accuracy: 0.7242 - lr: 3.3858e-06
Epoch 499/500
107/107 [==============================] - 105s 978ms/step - loss: 2.2615 - accuracy: 0.7365 - val_loss: 2.4878 - val_accuracy: 0.7230 - lr: 3.3519e-06
Epoch 500/500
107/107

In [10]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [12]:
# https://machinelearningmastery.com/save-load-keras-deep-learning-models/

model.save('/home/ordovas/IRONHACK/dice-scores-recognition/model_d6_augmented.h5')

In [13]:
model.save('/home/ordovas/IRONHACK/dice-scores-recognition/model_d6_augmented.h5')